# Get started with the Estimator primitive

Learn how to set up and use the Estimator primitive program.


## Overview

The `Estimator` primitive lets you efficiently calculate and interpret expectation values of quantum operators required for many algorithms. You can specify a list of circuits and observables, then evaluate expectation values and variances for a given parameter input.  


## Prepare the environment

1. Follow the steps in the [getting started guide](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/getting_started.html) to get your quantum service instance ready to use.

2. You'll need at least one circuit to submit to the program. Our examples all have circuits in them, but if you want to submit your own circuit, you can use Qiskit to create one. To learn how to create circuits by using Qiskit, see the [Circuit basics tutorial](https://qiskit.org/documentation/tutorials/circuits/01_circuit_basics.html).

3. Create a list of observables. Observables let you define the properties of the circuit that are relevant to your problem and enable you to efficiently measure their expectation value. For simplicity, you can use the [PauliSumOp class](https://qiskit.org/documentation/stubs/qiskit.opflow.primitive_ops.html#module-qiskit.opflow.primitive_ops) in Qiskit to define them, as illustrated in the example below.


## Start a session

A Qiskit Runtime **session** allows you to group a collection of iterative calls to the quantum computer. A session is started when the first job within the session is started. Subsequent jobs within the session are prioritized by the scheduler to minimize artificial delay within an iterative algorithm. Data used within a session, such as transpiled circuits, is also cached to avoid unnecessary overhead.

### Create an Estimator instance

The `Estimator` class takes in a `session` object that represents the session to use, and an `options` variable to control the execution environment.

`options` can be either a dictionary or a `qiskit_ibm_runtime.Options` class instance. Initializing it as an `Options` class allows you to do auto-complete. 

Some of the settings you can specify using `options`:

* **backend**: Optional string name of backend. If not specified a backend will be selected
    automatically (IBM Cloud only).
* **log_level**: logging level to set in the execution environment. The valid
        log levels are: `DEBUG`, `INFO`, `WARNING`, `ERROR`, and `CRITICAL`.
        The default level is `WARNING`.
* **optimization_level**: How much optimization to perform on the circuits. The default is 1.
* **resilience_level**: How much resilience to build against errors. Higher levels generate more accurate results, at the expense of longer processing times.

You can find more details in [the Options API reference](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/stubs/qiskit_ibm_runtime.Options.html).

With these arguments, you can create a `Estimator` instance with the desired options. Example:

In [1]:
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Estimator, Options

service = QiskitRuntimeService()
options = Options(optimization_level=1)

with Session(service=service, backend="ibmq_qasm_simulator"):
    estimator = Estimator(options=options)

## Invoke the Estimator primitive

You can invoke the `Estimator` primitive program using the `run()` method of the `Estimator` instance you just created. The method returns a `IBMRuntimeJob` instance which you can use to query for things like job ID and status. The `result()` method of the job will return the result. 

You can invoke the `run()` method multiple times with the different inputs within a session.

### Specify program inputs

The `Estimator.run()` method takes in the following arguments:

- **circuits**: A list of (parameterized) circuits that you want to investigate.
- **observables**: A list of observables to measure the expectation values.
- **parameters**: A list of parameters for the parameterized circuits. It should be omitted if the circuits provided are not parameterized.
- **parameter_values**: An optional list of concrete parameters to be bound.
- **kwargs**: Additional options to overwrite the default values. 

You can find more details in [the Estimator API reference](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/stubs/qiskit_ibm_runtime.Estimator.html).

### Run the job & print results

Run the job, specifying your previously defined inputs and options.  Use `circuits`, `observables`, and `parameter_values` to use a specific parameter and observable with the specified circuit.

For example, this line `psi1_H23_result = estimator.run(circuits=[psi1, psi1], observables=[H2, H3], parameter_values=[theta1]*2).result()` specifies the following:

- Return the value for using observable `H2` and parameter `theta1` with the circuit `psi1`.
- Return the value for using observable `H3` and parameter `theta1` with the circuit `psi1`.


In [ ]:
# Prepare the inputs

from qiskit.circuit.library import RealAmplitudes
from qiskit.quantum_info import SparsePauliOp

psi1 = RealAmplitudes(num_qubits=2, reps=2)
psi2 = RealAmplitudes(num_qubits=2, reps=3)

H1 = SparsePauliOp.from_list([("II", 1), ("IZ", 2), ("XI", 3)])
H2 = SparsePauliOp.from_list([("IZ", 1)])
H3 = SparsePauliOp.from_list([("ZI", 1), ("ZZ", 1)])

theta1 = [0, 1, 1, 2, 3, 5]
theta2 = [0, 1, 1, 2, 3, 5, 8, 13]
theta3 = [1, 2, 3, 4, 5, 6]

In [ ]:
with Session(service=service):
    estimator = Estimator(options=options)
    
    # calculate [ <psi1(theta1)|H1|psi1(theta1)> ]
    psi1_H1 = estimator.run(circuits=[psi1], observables=[H1], parameter_values=[theta1])
    print(psi1_H1.result())

    # calculate [ <psi1(theta1)|H2|psi1(theta1)>, <psi1(theta1)|H3|psi1(theta1)> ]
    psi1_H23 = estimator.run(circuits=[psi1, psi1], observables=[H2, H3], parameter_values=[theta1]*2)
    print(psi1_H23.result())

    # calculate [ <psi2(theta2)|H2|psi2(theta2)> ]
    psi2_H2 = estimator.run(circuits=[psi2], observables=[H2], parameter_values=[theta2])
    print(psi2_H2.result())

    # calculate [ <psi1(theta1)|H1|psi1(theta1)>, <psi1(theta3)|H1|psi1(theta3)> ]
    psi1_H1_job = estimator.run(circuits=[psi1, psi1], observables=[H1, H1], parameter_values=[theta1, theta3])
    print(psi1_H1_job.result())

    # calculate [ <psi1(theta1)|H1|psi1(theta1)>,
    #             <psi2(theta2)|H2|psi2(theta2)>,
    #             <psi1(theta3)|H3|psi1(theta3)> ]
    psi12_H23 = estimator.run(circuits=[psi1, psi2, psi1], observables=[H1, H2, H3], parameter_values=[theta1, theta2, theta3])
    print(psi12_H23.result())

EstimatorResult(values=array([1.59570312]), metadata=[{'variance': 8.661632537841797, 'shots': 1024}])
EstimatorResult(values=array([-0.5390625 ,  0.07421875]), metadata=[{'variance': 0.70941162109375, 'shots': 1024}, {'variance': 1.9894332885742188, 'shots': 1024}])
EstimatorResult(values=array([0.17773438]), metadata=[{'variance': 0.9684104919433594, 'shots': 1024}])


The results align with the parameter - circuit - observable tuples specified previously.  For example, the first result: `EstimatorResult(values=array([1.53710938]), metadata=[{'variance': 9.249675750732422, 'shots': 1024}])` is the output of the parameter labeled `theta1` and observable `H1` being sent to the first circuit.